# SCA 7216 Homework 4 Question 2 on Radiosonde Observation Removal
## 1. Problem setup

##### 1.1 Imports and problem constants

In [1]:
import numpy as np 
import pandas as pd
import geopy.distance
import itertools

In [2]:
# Station names and locations
stations = pd.DataFrame(
        np.array([['KALY', 42.75, -73.88],
                  ['KBUF', 42.93, -78.73],
                  ['KPIT', 40.50, -80.27],
                  ['KOKX', 40.87, -72.87],
                  ['KGYX', 43.88, -70.25],
                  ['CWMW', 46.25, -76.00],
                  ['KCAR', 46.87, -68.02]]),
        columns=['Station_ID', 'Latitude', 'Longitude']
    )
stations.set_index('Station_ID', inplace=True)
coords = list(zip(stations.Latitude.values,stations.Longitude.values))
stations['Coordinates'] = coords
observations = stations # Control case
stations

,Latitude,Longitude,Coordinates
Station_ID,,,
KALY,42.75,-73.88,"(42.75, -73.88)"
KBUF,42.93,-78.73,"(42.93, -78.73)"
KPIT,40.5,-80.27,"(40.5, -80.27)"
KOKX,40.87,-72.87,"(40.87, -72.87)"
KGYX,43.88,-70.25,"(43.88, -70.25)"
CWMW,46.25,-76.0,"(46.25, -76.0)"
KCAR,46.87,-68.02,"(46.87, -68.02)"


##### 1.2 - Given covariances

In [3]:
# Variances given in the question in the form {pressure_level,variance_value}
sigma2_b = pd.DataFrame.from_dict({950:1.0, 500:1.5}, orient='index', columns=['Value']) # background variance
sigma2_o = pd.DataFrame.from_dict({950:1.2, 500:0.4}, orient='index', columns=['Value']) # observation variance
sigma2_b.index.name = 'Pressure' 
sigma2_o.index.name = 'Pressure' 
sigma2_b

,Value
Pressure,
950,1.0
500,1.5


In [4]:
sigma2_o

,Value
Pressure,
950,1.2
500,0.4


In [5]:
def gaussian_corr(dist,Lh = 250.):
    """ 
    Correlation model (Gaussian) for a given distance between two locations. 

    Parameters
        dist - float (or array of type float) chordals distance between correlated points in km
        Lh   - float correlation length in km

    Returns 
        Correlation value - float
    """
    return np.exp(-dist**2/(2*Lh**2))

##### 1.3 - Distances between stations 

In [6]:
# Nested dictionary to represent the matrix of distances between stations
distances = dict.fromkeys(stations.index) 

# Cycle each distinct pair of stations
for i,j in itertools.combinations(stations.index,2):

    # Distance in km between each distinct Latitude Longitude pair
    d = geopy.distance.geodesic(stations.loc[i].Coordinates,stations.loc[j].Coordinates).km

    # And by symmetry
    if distances[i] == None:
        distances[i] = dict.fromkeys(stations.index,0)
    if distances[j] == None:
        distances[j] = dict.fromkeys(stations.index,0)
    
    distances[i][j] = d
    distances[j][i] = d
    
distances = pd.DataFrame.from_dict(distances)
distances

,KALY,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,0.000000,396.946922,588.019540,225.041346,320.083677,423.857299,651.197575
KBUF,396.946922,0.000000,298.762132,537.240228,694.648387,427.841394,951.381396
KPIT,588.019540,298.762132,0.000000,626.714903,908.046776,726.261389,1212.821157
KOKX,225.041346,537.240228,626.714903,0.000000,397.889078,648.893813,771.911012
KGYX,320.083677,694.648387,908.046776,397.889078,0.000000,523.685886,375.378853
CWMW,423.857299,427.841394,726.261389,648.893813,523.685886,0.000000,615.488323
KCAR,651.197575,951.381396,1212.821157,771.911012,375.378853,615.488323,0.000000


##### 1.4 - Declaration of covariance matrices

In [7]:
C = gaussian_corr(distances)
C

,KALY,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,1.000000,0.283502,0.062905,0.666878,0.440595,0.237583,0.033626
KBUF,0.283502,1.000000,0.489647,0.099359,0.021062,0.231220,0.000717
KPIT,0.062905,0.489647,1.000000,0.043189,0.001365,0.014704,0.000008
KOKX,0.666878,0.099359,0.043189,1.000000,0.281809,0.034441,0.008508
KGYX,0.440595,0.021062,0.001365,0.281809,1.000000,0.111473,0.323915
CWMW,0.237583,0.231220,0.014704,0.034441,0.111473,1.000000,0.048286
KCAR,0.033626,0.000717,0.000008,0.008508,0.323915,0.048286,1.000000


In [8]:
I = pd.DataFrame(np.eye(7),index=distances.index,columns=distances.columns)
I

,KALY,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,1.0,0.0,0.0,0.0,0.0,0.0,0.0
KBUF,0.0,1.0,0.0,0.0,0.0,0.0,0.0
KPIT,0.0,0.0,1.0,0.0,0.0,0.0,0.0
KOKX,0.0,0.0,0.0,1.0,0.0,0.0,0.0
KGYX,0.0,0.0,0.0,0.0,1.0,0.0,0.0
CWMW,0.0,0.0,0.0,0.0,0.0,1.0,0.0
KCAR,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
# The background error covariance does not change between the two experiments
B = {500:sigma2_b.loc[500].values[0] + (I-I),950:sigma2_b.loc[950].values[0] + (I-I)}
B[500]

,KALY,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,1.5,1.5,1.5,1.5,1.5,1.5,1.5
KBUF,1.5,1.5,1.5,1.5,1.5,1.5,1.5
KPIT,1.5,1.5,1.5,1.5,1.5,1.5,1.5
KOKX,1.5,1.5,1.5,1.5,1.5,1.5,1.5
KGYX,1.5,1.5,1.5,1.5,1.5,1.5,1.5
CWMW,1.5,1.5,1.5,1.5,1.5,1.5,1.5
KCAR,1.5,1.5,1.5,1.5,1.5,1.5,1.5


In [10]:
B[950]

,KALY,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,1.0,1.0,1.0,1.0,1.0,1.0,1.0
KBUF,1.0,1.0,1.0,1.0,1.0,1.0,1.0
KPIT,1.0,1.0,1.0,1.0,1.0,1.0,1.0
KOKX,1.0,1.0,1.0,1.0,1.0,1.0,1.0
KGYX,1.0,1.0,1.0,1.0,1.0,1.0,1.0
CWMW,1.0,1.0,1.0,1.0,1.0,1.0,1.0
KCAR,1.0,1.0,1.0,1.0,1.0,1.0,1.0


##### 1.5 - Functions for linear analysis error optimisation

In [11]:
def clean(df):
    """ 
    Gets rid of full rows and columns of NaN in a particular DataFrame

    Parameters:  
        df -  pandas.core.frame.DataFrame to be cleaned
    Returns: 
        Cleaned df
    """ 
    return df.dropna(axis=0,how='all').dropna(axis=1,how='all')

def inv_matrix_pandas(df):
    """
    Takes an invertible matrix df and returns its inverse. 
    
    Parameters:
        df - pandas.core.frame.DataFrame
    Returns:
        Matrix inverse of df from numpy linalg inverse changed back to pandas
    """ 
    return clean(pd.DataFrame(np.linalg.pinv(df.values), df.columns, df.index))

def kalman_gain():
    """ 
    Calculates the Kalman gain at 500 and 950hPa with the given state of B, H, and R

    Returns
        Dictionary of Kalman gain matrices hashed by the pressure level in hPa
    """ 
    K = {}
    for level in [500,950]:
        K[level] = B[level]*H.transpose()*inv_matrix_pandas(clean(H*B[level]*H.transpose()) + clean(R[level]))
        
    return K

def analysis_error_covariance():
    """ 
    Calculates the analysis error covariance at 500 and 950hPa with the given state of B, H, and R

    Returns
        Dictionary of analysis covariance matrices hashed by the pressure level in hPa
    """ 
    A = {} 
    K = kalman_gain()

    for level in [500,950]:
        A[level] = clean((I-K[level]*H)*B[level])
    return A

##### 1.6 - Values for observation operator and covariance matrices before removal of station

In [12]:
H = I
H

,KALY,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,1.0,0.0,0.0,0.0,0.0,0.0,0.0
KBUF,0.0,1.0,0.0,0.0,0.0,0.0,0.0
KPIT,0.0,0.0,1.0,0.0,0.0,0.0,0.0
KOKX,0.0,0.0,0.0,1.0,0.0,0.0,0.0
KGYX,0.0,0.0,0.0,0.0,1.0,0.0,0.0
CWMW,0.0,0.0,0.0,0.0,0.0,1.0,0.0
KCAR,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
R = {500:sigma2_o.loc[500].values[0]*C,950:sigma2_o.loc[950].values[0]*C}
R[500]

,KALY,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,0.400000,0.113401,0.025162,0.266751,0.176238,0.095033,0.013450
KBUF,0.113401,0.400000,0.195859,0.039744,0.008425,0.092488,0.000287
KPIT,0.025162,0.195859,0.400000,0.017276,0.000546,0.005882,0.000003
KOKX,0.266751,0.039744,0.017276,0.400000,0.112724,0.013776,0.003403
KGYX,0.176238,0.008425,0.000546,0.112724,0.400000,0.044589,0.129566
CWMW,0.095033,0.092488,0.005882,0.013776,0.044589,0.400000,0.019315
KCAR,0.013450,0.000287,0.000003,0.003403,0.129566,0.019315,0.400000


In [14]:
R[950]

,KALY,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,1.200000,0.340203,0.075486,0.800253,0.528714,0.285099,0.040351
KBUF,0.340203,1.200000,0.587576,0.119231,0.025274,0.277464,0.000860
KPIT,0.075486,0.587576,1.200000,0.051827,0.001638,0.017645,0.000009
KOKX,0.800253,0.119231,0.051827,1.200000,0.338171,0.041329,0.010209
KGYX,0.528714,0.025274,0.001638,0.338171,1.200000,0.133767,0.388698
CWMW,0.285099,0.277464,0.017645,0.041329,0.133767,1.200000,0.057944
KCAR,0.040351,0.000860,0.000009,0.010209,0.388698,0.057944,1.200000


##### 1.7 - Analysis before removal of stations

In [15]:
analyses = [] 
A = {}
A = analysis_error_covariance()
analyses.append(A) # For comparison with other analyses later
A[500]

,KALY,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,0.276561,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
KBUF,0.000000,0.296233,0.000000,0.000000,0.000000,0.000000,0.000000
KPIT,0.000000,0.000000,0.302958,0.000000,0.000000,0.000000,0.000000
KOKX,0.000000,0.000000,0.000000,0.289057,0.000000,0.000000,0.000000
KGYX,0.000000,0.000000,0.000000,0.000000,0.296919,0.000000,0.000000
CWMW,0.000000,0.000000,0.000000,0.000000,0.000000,0.309795,0.000000
KCAR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.310162


In [16]:
A[950]

,KALY,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,0.435463,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
KBUF,0.000000,0.492394,0.000000,0.000000,0.000000,0.000000,0.000000
KPIT,0.000000,0.000000,0.510105,0.000000,0.000000,0.000000,0.000000
KOKX,0.000000,0.000000,0.000000,0.472531,0.000000,0.000000,0.000000
KGYX,0.000000,0.000000,0.000000,0.000000,0.498204,0.000000,0.000000
CWMW,0.000000,0.000000,0.000000,0.000000,0.000000,0.530679,0.000000
KCAR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.530235


## 2. After removal of Albany NY Station:

##### 2.1 - Removal's update to observation operator and observation covariance matrices

In [17]:
# Update H and R on removing the Albany station 
H = I.drop('KALY',axis=1) # Removes radiosonde from the column of the observation operator
for level in [500,950]: # Removes radiosonde from both axes in the observation covariance matrix
    R[level] = R[level].drop('KALY', axis=0).drop('KALY', axis=1)
H

,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KALY,0.0,0.0,0.0,0.0,0.0,0.0
KBUF,1.0,0.0,0.0,0.0,0.0,0.0
KPIT,0.0,1.0,0.0,0.0,0.0,0.0
KOKX,0.0,0.0,1.0,0.0,0.0,0.0
KGYX,0.0,0.0,0.0,1.0,0.0,0.0
CWMW,0.0,0.0,0.0,0.0,1.0,0.0
KCAR,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
R[500]

,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KBUF,0.400000,0.195859,0.039744,0.008425,0.092488,0.000287
KPIT,0.195859,0.400000,0.017276,0.000546,0.005882,0.000003
KOKX,0.039744,0.017276,0.400000,0.112724,0.013776,0.003403
KGYX,0.008425,0.000546,0.112724,0.400000,0.044589,0.129566
CWMW,0.092488,0.005882,0.013776,0.044589,0.400000,0.019315
KCAR,0.000287,0.000003,0.003403,0.129566,0.019315,0.400000


In [19]:
R[950]

,KBUF,KPIT,KOKX,KGYX,CWMW,KCAR
KBUF,1.200000,0.587576,0.119231,0.025274,0.277464,0.000860
KPIT,0.587576,1.200000,0.051827,0.001638,0.017645,0.000009
KOKX,0.119231,0.051827,1.200000,0.338171,0.041329,0.010209
KGYX,0.025274,0.001638,0.338171,1.200000,0.133767,0.388698
CWMW,0.277464,0.017645,0.041329,0.133767,1.200000,0.057944
KCAR,0.000860,0.000009,0.010209,0.388698,0.057944,1.200000


##### 2.2 - Effect on analysis error covariance

In [20]:
A = analysis_error_covariance()
analyses.append(A) # For comparison with other analyses later
A[500]

,CWMW,KBUF,KCAR,KGYX,KOKX,KPIT
CWMW,0.312212,0.00000,0.000000,0.000000,0.000000,0.000000
KBUF,0.000000,0.29976,0.000000,0.000000,0.000000,0.000000
KCAR,0.000000,0.00000,0.310162,0.000000,0.000000,0.000000
KGYX,0.000000,0.00000,0.000000,0.305445,0.000000,0.000000
KOKX,0.000000,0.00000,0.000000,0.000000,0.311005,0.000000
KPIT,0.000000,0.00000,0.000000,0.000000,0.000000,0.303007


In [21]:
A[950]

,CWMW,KBUF,KCAR,KGYX,KOKX,KPIT
CWMW,0.535988,0.000000,0.000000,0.000000,0.000000,0.000000
KBUF,0.000000,0.501157,0.000000,0.000000,0.000000,0.000000
KCAR,0.000000,0.000000,0.530431,0.000000,0.000000,0.000000
KGYX,0.000000,0.000000,0.000000,0.517747,0.000000,0.000000
KOKX,0.000000,0.000000,0.000000,0.000000,0.532839,0.000000
KPIT,0.000000,0.000000,0.000000,0.000000,0.000000,0.510138


## 3. After removal of Gray au Maine Station:

##### 3.1 - Removal's update to observation operator and observation covariance matrices

In [22]:
# Update H and R on removing the Gray au Maine station 
H = H.drop('KGYX',axis=1) # Removes radiosonde from the column of the observation operator
for level in [500,950]: # Removes radiosonde from both axes in the observation covariance matrix
    R[level] = R[level].drop('KGYX', axis=0).drop('KGYX', axis=1)
H

,KBUF,KPIT,KOKX,CWMW,KCAR
KALY,0.0,0.0,0.0,0.0,0.0
KBUF,1.0,0.0,0.0,0.0,0.0
KPIT,0.0,1.0,0.0,0.0,0.0
KOKX,0.0,0.0,1.0,0.0,0.0
KGYX,0.0,0.0,0.0,0.0,0.0
CWMW,0.0,0.0,0.0,1.0,0.0
KCAR,0.0,0.0,0.0,0.0,1.0


In [23]:
R[500]

,KBUF,KPIT,KOKX,CWMW,KCAR
KBUF,0.400000,0.195859,0.039744,0.092488,0.000287
KPIT,0.195859,0.400000,0.017276,0.005882,0.000003
KOKX,0.039744,0.017276,0.400000,0.013776,0.003403
CWMW,0.092488,0.005882,0.013776,0.400000,0.019315
KCAR,0.000287,0.000003,0.003403,0.019315,0.400000


In [24]:
R[950]

,KBUF,KPIT,KOKX,CWMW,KCAR
KBUF,1.200000,0.587576,0.119231,0.277464,0.000860
KPIT,0.587576,1.200000,0.051827,0.017645,0.000009
KOKX,0.119231,0.051827,1.200000,0.041329,0.010209
CWMW,0.277464,0.017645,0.041329,1.200000,0.057944
KCAR,0.000860,0.000009,0.010209,0.057944,1.200000


##### 3.2 - Effect on analysis error covariance

In [25]:
A = analysis_error_covariance()
analyses.append(A) # For comparison with other analyses later
A[500]

,CWMW,KBUF,KCAR,KOKX,KPIT
CWMW,0.312803,0.000000,0.000000,0.000000,0.000000
KBUF,0.000000,0.299766,0.000000,0.000000,0.000000
KCAR,0.000000,0.000000,0.315663,0.000000,0.000000
KOKX,0.000000,0.000000,0.000000,0.315164,0.000000
KPIT,0.000000,0.000000,0.000000,0.000000,0.303008


In [26]:
A[950]

,CWMW,KBUF,KCAR,KOKX,KPIT
CWMW,0.537381,0.000000,0.000000,0.000000,0.000000
KBUF,0.000000,0.501161,0.000000,0.000000,0.000000
KCAR,0.000000,0.000000,0.545126,0.000000,0.000000
KOKX,0.000000,0.000000,0.000000,0.543997,0.000000
KPIT,0.000000,0.000000,0.000000,0.000000,0.510141


##  4. Discussion on differences in analysis covariance

##### 4.1 - Raw changes in the analysis covariance matrix

At 500 hPa there was an increase in the analysis error covariance between the base case, and after the removal of a station. This most notably affected the reliability of particular stations within themselves.

In [27]:
clean(analyses[1][500] - analyses[0][500]) # The increase in analysis covariance on deleting a measurement at 500 hPa

,CWMW,KBUF,KCAR,KGYX,KOKX,KPIT
CWMW,0.002417,0.000000,0.000000e+00,0.000000,0.000000,0.000000
KBUF,0.000000,0.003528,0.000000e+00,0.000000,0.000000,0.000000
KCAR,0.000000,0.000000,5.765427e-07,0.000000,0.000000,0.000000
KGYX,0.000000,0.000000,0.000000e+00,0.008526,0.000000,0.000000
KOKX,0.000000,0.000000,0.000000e+00,0.000000,0.021948,0.000000
KPIT,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000049


The effect is also visible at the 950 hPa level to a stronger degree given the stronger observation variance.

In [28]:
clean(analyses[1][950] - analyses[0][950]) # The increase in analysis covariance on deleting a measurement at 950 hPa

,CWMW,KBUF,KCAR,KGYX,KOKX,KPIT
CWMW,0.005309,0.000000,0.000000,0.000000,0.000000,0.000000
KBUF,0.000000,0.008763,0.000000,0.000000,0.000000,0.000000
KCAR,0.000000,0.000000,0.000196,0.000000,0.000000,0.000000
KGYX,0.000000,0.000000,0.000000,0.019543,0.000000,0.000000
KOKX,0.000000,0.000000,0.000000,0.000000,0.060307,0.000000
KPIT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000032


Deleting two measurements increases the analysis error covariance much more dramatically at both levels.

In [29]:
clean(analyses[2][500] - analyses[0][500]) # The increase in analysis covariance on deleting two measurements at 500 hPa

,CWMW,KBUF,KCAR,KOKX,KPIT
CWMW,0.003008,0.000000,0.000000,0.000000,0.000000
KBUF,0.000000,0.003533,0.000000,0.000000,0.000000
KCAR,0.000000,0.000000,0.005502,0.000000,0.000000
KOKX,0.000000,0.000000,0.000000,0.026107,0.000000
KPIT,0.000000,0.000000,0.000000,0.000000,0.000049


In [30]:
diff_ex = clean(analyses[2][950] - analyses[0][950]) # The increase in analysis covariance on deleting two measurements at 500 hPa
diff_ex # A variable used for discussion later

,CWMW,KBUF,KCAR,KOKX,KPIT
CWMW,0.006702,0.000000,0.000000,0.000000,0.000000
KBUF,0.000000,0.008767,0.000000,0.000000,0.000000
KCAR,0.000000,0.000000,0.014891,0.000000,0.000000
KOKX,0.000000,0.000000,0.000000,0.071466,0.000000
KPIT,0.000000,0.000000,0.000000,0.000000,0.000035


##### 4.2 - Discussion

Évidemment l'enlevement d'une station n'affecte les autres. Un rapide coup d'oeil à l'ensemble de données permet d'en com prendre l'impact. Par exemple, à une pression de 950 hPa, la variance d'erreur d'analyse à la station 'KOKX' a augmenté de plus de 600 % avec l'enlevement des deux stations! Voir ci-dessous:

In [31]:
print(f'{np.round(analyses[0][950]['KOKX']['KOKX']/diff_ex['KOKX']['KOKX'] * 100)}%')

661.0%


La capacité des modèles numériques à réaliser des prédictions précises dépend de la disponibilité et de l'intégrité des données. Cet exercice montre clairement que la suppression de deux stations peut augmenter considérablement l'erreur d'analyse à certaines stations, et généralement à toutes les stations comme on peut voir dans les chiffres des covariances d'analyses.